In [ ]:
import requests
import pandas as pd
import plotly.express as px
import streamlit as st
import unicodedata

# -----------------------------------------------------------------------------
# 1. CONFIGURACIÓN Y CONTEXTO
# -----------------------------------------------------------------------------
# Recuperamos variables del contexto de app.py
estado_seleccionado = locals().get("ESTADO_SELECCIONADO", "Nuevo León")
palette = locals().get("active_palette", ["#889064", "#0576F3"])
active_font = locals().get("active_font", "sans-serif")

# Usaremos el primer color de la paleta para las barras
bar_color = palette[0]

# -----------------------------------------------------------------------------
# 2. FUNCIONES AUXILIARES
# -----------------------------------------------------------------------------

def normalize_text(text):
    """Elimina acentos y pasa a mayúsculas para comparar nombres de estados."""
    if not isinstance(text, str): return str(text)
    text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode("utf-8")
    return text.upper().strip()

@st.cache_data(show_spinner=False)
def fetch_inversion_data():
    """Descarga los datos de la API de Economía (se cachea para velocidad)."""
    url = "https://www.economia.gob.mx/apidatamexico/tesseract/data.jsonrecords"
    params = {
        "cube": "Anuncios_Inversion_BaseInversion",
        "drilldowns": "Quarter,Country,State,IA_Sector,IA_Sociedad",
        "locale": "es",
        "measures": "Monto Inversion"
    }
    try:
        response = requests.get(url, params=params, timeout=20)
        response.raise_for_status()
        data = response.json()
        return pd.DataFrame(data.get("data", []))
    except Exception as e:
        st.error(f"Error al conectar con la Secretaría de Economía: {e}")
        return pd.DataFrame()

# -----------------------------------------------------------------------------
# 3. LÓGICA PRINCIPAL
# -----------------------------------------------------------------------------

if not estado_seleccionado:
    st.info("Selecciona un estado para ver los anuncios de inversión.")
else:
    with st.spinner(f"Consultando anuncios de inversión para {estado_seleccionado}..."):

        # 1. Obtener datos
        df = fetch_inversion_data()

        if not df.empty:
            # 2. Normalizar nombres para el filtrado robusto
            # Creamos una columna temporal normalizada para filtrar
            df["State_Norm"] = df["State"].apply(normalize_text)
            estado_target = normalize_text(estado_seleccionado)

            # 3. Filtrar por Estado
            df_estado = df[df["State_Norm"] == estado_target].copy()

            if not df_estado.empty:
                # 4. Extraer el año más reciente disponible para ese estado
                # El formato de Quarter suele ser "2024-Q1" o similar
                df_estado["Anio"] = df_estado["Quarter"].astype(str).str.split("-").str[0]
                anio_max = df_estado["Anio"].max()

                # Filtrar solo el último año
                df_estado = df_estado[df_estado["Anio"] == anio_max]

                # 5. Agrupar por Sector y sumar Monto
                df_grouped = df_estado.groupby("IA_Sector", as_index=False)["Monto Inversion"].sum()

                # Top 10
                df_top10 = df_grouped.sort_values(by="Monto Inversion", ascending=False).head(10)

                # Convertir a Millones de USD (o MXN según la fuente, usualmente es USD en anuncios)
                # Asumiremos Millones para visualización limpia
                df_top10["Monto_M"] = df_top10["Monto Inversion"] / 1_000_000

                # Ordenar para que la barra mayor quede arriba en gráfica horizontal
                df_top10 = df_top10.sort_values(by="Monto_M", ascending=True)

                if not df_top10.empty:
                    # 6. Crear gráfica
                    fig = px.bar(
                        df_top10,
                        x="Monto_M",
                        y="IA_Sector",
                        orientation="h",
                        text="Monto_M",
                        title=f"Top Inversiones por Industria: {estado_seleccionado} ({anio_max})",
                        labels={"Monto_M": "Millones de Dólares (Estimado)", "IA_Sector": ""}
                    )

                    # Estilos
                    fig.update_traces(
                        marker_color=bar_color,
                        texttemplate="$%{text:,.1f} M",
                        textposition="outside"
                    )

                    fig.update_layout(
                        font_family=active_font,
                        title_font_size=18,
                        title_x=0, # Alineación izquierda suele verse mejor en web
                        plot_bgcolor="white",
                        paper_bgcolor="white",
                        xaxis=dict(showgrid=True, gridcolor="#eee", title="Monto de Inversión (Millones)"),
                        yaxis=dict(showgrid=False),
                        margin=dict(l=20, r=50, t=50, b=50),
                        height=500
                    )

                    st.plotly_chart(fig, use_container_width=True)
                    st.caption("Fuente: DataMéxico / Secretaría de Economía (Anuncios de Inversión).")
                else:
                    st.warning(f"Se encontraron registros para {anio_max}, pero los montos son cero o no válidos.")
            else:
                st.warning(f"No se encontraron anuncios de inversión registrados recientemente para {estado_seleccionado}.")
        else:
            st.warning("No se pudo descargar la base de datos de inversión.")